https://www.youtube.com/watch?v=tcqEUSNCn8I
https://github.com/pixegami/langchain-rag-tutorial/blob/main/query_data.py

In [1]:
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
import os
import shutil
from langchain_community.llms import Ollama
import json

from langchain.chains import RetrievalQA
from langchain_core.prompts import ChatPromptTemplate

/home/deborah/FS24/masterarbeit/State_of_the_art/.venv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
paper_id = "2402.01383v1"
CHROMA_PATH = paper_id+"/chroma"
#CHROMA_PATH = paper_id+"/chroma300"
DATA_PATH = paper_id
file = 'rag_largechroma'
#file = 'rag300'

In [3]:

def store_summary(survey1,filename):
    with open(f'summaries/'+filename+'.txt', 'w') as file:
        file.write(survey1)
def write_json(documents_data,file_name):
    converted_data = [dict(item) for item in documents_data['source_documents']]
    dict_data = {'answer':documents_data['result'],'source':converted_data}
    with open('summaries/'+file_name+'.json', 'w') as file:
        json.dump(dict_data, file, indent=4)


### Create Chroma database

In [6]:

def load_documents():
    loader = DirectoryLoader(DATA_PATH, glob="*.pdf")
    documents = loader.load()
    return documents

In [7]:
documents = load_documents()

In [8]:
documents[4].metadata

{'source': '2402.01383v1/2307.07889v3.LLM_Comparative_Assessment__Zero_shot_NLG_Evaluation_through_Pairwise_Comparisons_using_Large_Language_Models.pdf'}

In [9]:
def split_text(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,         #300,
        chunk_overlap=100,
        length_function=len,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

    document = chunks[10]
    print(document.page_content)
    print(document.metadata)

    return chunks

In [10]:
chunks = split_text(documents)

Split 45 documents into 5364 chunks.
In the course of our investigation (Table 1 and Section 3), we uncover intriguing findings related to the assessment of answer quality. Firstly, we observe hesitancy among humans, including both crowd-sourced and expert annotators, in determin- ing answer quality, while LLMs exhibit greater cer- tainty in their evaluations. Furthermore, we notice a substantial difference in fact-checking capabil- ities between human judges and LLMs. Human judges generally do not thoroughly fact-check an- swers unless the factual error is glaringly evident, whereas LLMs demonstrate some degree of fact- checking ability, albeit with imperfections. An- other significant finding is that both human judges

2In this work, we refer to different settings in answering questions as different models, although all the answers are generated by GPT-4.
{'source': '2402.01383v1/2307.03025v3.Style_Over_Substance__Evaluation_Biases_for_Large_Language_Models.pdf', 'start_index': 6749}

In [11]:
#
some = chunks[0:10]
#print(some)
print(len(chunks[11].page_content))
chunks[11].page_content

857


'Our proposed approach requires human judges and LLMs to evaluate the machine-generated text independently from three aspects: “Accuracy”, “Helpfulness”, and “Language”. This allows us to achieve a more comprehensive and transparent understanding of the quality of machine-generated text. Our empirical findings demonstrate a sig- nificant improvement in the evaluation quality of GPT-4, particularly in terms of factual accuracy. However, we observe that humans still exhibit in- decisiveness in their assessments.\n\nBased on our findings, we highly recommend that practitioners evaluate machine-generated text from various perspectives rather than depending solely on a single unified measure. Additionally, we advise practitioners to exercise caution when using crowd-sourced human annotators to assess the performance of LLMs.\n\nModel type\n\nOutput\n\nCorrect'

In [12]:
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

/home/deborah/FS24/masterarbeit/State_of_the_art/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [13]:
def save_to_chroma(chunks: list[Document]):
    # Clear out the database first.
    #if os.path.exists(CHROMA_PATH):
     #   shutil.rmtree(CHROMA_PATH)


    # Ensure the directory exists and has the correct permissions
    #os.makedirs(CHROMA_PATH, exist_ok=True)
    #os.chmod(CHROMA_PATH, 0o755)
        
    # Create a new DB from the documents.
    db = Chroma.from_documents(
        chunks, embedding_function, persist_directory=CHROMA_PATH
    )
    db.persist()
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")


In [14]:
save_to_chroma(chunks)

Saved 5364 chunks to 2402.01383v1/chroma.


/home/deborah/FS24/masterarbeit/State_of_the_art/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


### Use database

In [5]:
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

/home/deborah/FS24/masterarbeit/State_of_the_art/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/deborah/FS24/masterarbeit/State_of_the_art/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

⚠️ It looks like you upgraded from a version below 0.6 and could benefit from vacuuming your database. Run chromadb utils vacuum --help for more information.


In [17]:
llm = Ollama(model = "mistral",temperature=0)
system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
)

rag = RetrievalQA.from_chain_type(
            llm=llm,
            retriever=db.as_retriever(),
            return_source_documents = True,
            
            #memory=ConversationSummaryMemory(llm = Ollama(model="mistral")),
            #chain_type_kwargs={"prompt": pt, "verbose": True},
        )

answer = rag.invoke("What is the recent development of LLM generated text evaluation?")
rag_overview = answer['result']

KeyboardInterrupt: 

In [20]:
print(rag_overview)
store_summary(rag_overview,'rag_overview300')

 Recent developments in LLM (Large Language Model) generated text evaluation include using LLMs to evaluate the quality of their own text outputs as an alternative to human evaluations. This approach can provide a more comprehensive understanding of LLM performance and predict how a tuned version might behave in various scenarios. Additionally, applying LLMs as evaluators for text generation has been shown to be scalable, cost-effective, and can significantly enhance the ability to assess the quality of generated text. For instance, Fu et al. (2023) use LLM's predicted text probability as the automated score. Furthermore, this approach has been applied to evaluate Chinese LLMs, resulting in a faster evaluation process and decreased average annotation cost per sample. However, it is important to note that the utilization of LLMs as evaluators for text generation is still in the exploratory phase, and there are limitations that provide opportunities for future work, such as the performan

In [21]:
a = answer['source_documents'][0]
converted_data = [dict(item) for item in answer['source_documents']]
#dict(answer['source_documents'])
converted_data
answer

{'query': 'What is the recent development of LLM generated text evaluation?',
 'result': " Recent developments in LLM (Large Language Model) generated text evaluation include using LLMs to evaluate the quality of their own text outputs as an alternative to human evaluations. This approach can provide a more comprehensive understanding of LLM performance and predict how a tuned version might behave in various scenarios. Additionally, applying LLMs as evaluators for text generation has been shown to be scalable, cost-effective, and can significantly enhance the ability to assess the quality of generated text. For instance, Fu et al. (2023) use LLM's predicted text probability as the automated score. Furthermore, this approach has been applied to evaluate Chinese LLMs, resulting in a faster evaluation process and decreased average annotation cost per sample. However, it is important to note that the utilization of LLMs as evaluators for text generation is still in the exploratory phase, a

In [45]:
write_json(answer,'rag_overview300')

In [48]:
type(answer)
sources = [doc.metadata.get("source", None) for doc, _score in answer]
sources

ValueError: too many values to unpack (expected 2)

### Change more parameters example

In [22]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)

In [26]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain



retriever=db.as_retriever(20)
llm = Ollama(model = "mistral",temperature=0)

system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Context: {context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, question_answer_chain)


TypeError: VectorStore.as_retriever() takes 1 positional argument but 2 were given

In [26]:
query = 'What is the recent development of LLM generated text evaluation?'
chain.invoke({"input": query})

TypeError: 'VectorStoreRetriever' object is not callable

### Other parameters
https://medium.com/@callumjmac/implementing-rag-in-langchain-with-chroma-a-step-by-step-guide-16fc21815339

In [6]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:
{context}
 - -
Answer the question based on the above context: {question}
Write your answer in about 2000 words.
"""

In [7]:
def query_rag(query_text,k=3):
  # Retrieving the context from the DB using similarity search
  results = db.similarity_search_with_relevance_scores(query_text, k=k)

  # Check if there are any matching results or if the relevance score is too low
  if len(results) == 0 or results[0][1] < 0.7:
    print(f"Unable to find matching results.")

  # Combine context from matching documents
  context_text = "\n\n - -\n\n".join([doc.page_content for doc, _score in results])
 
  # Create prompt template using context and query text
  prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
  prompt = prompt_template.format(context=context_text, question=query_text)

  llm = Ollama(model = "mistral",temperature=0)
  # Generate response text based on the prompt
  response_text = llm.invoke(prompt)
 
   # Get sources of the matching documents
  sources = [doc.metadata.get("source", None) for doc, _score in results]
 
  # Format and return response including generated text and sources
  formatted_response = f"Response: {response_text}\nSources: {sources} \nSourceText:{context_text}"
  return formatted_response, response_text



In [8]:
topic = "LLM-based NLG Evaluation: Current Status and Challenges"
db.similarity_search_with_relevance_scores(f'what are recent developments in {topic}?', k=2)

[(Document(metadata={'source': '2402.01383v1/2311.18702v2.CritiqueLLM__Towards_an_Informative_Critique_Generation_Model_for_Evaluation_of_Large_Language_Model_Generation.pdf', 'start_index': 6910}, page_content='2 Related Work\n\nEvaluation is a long-standing task in NLP, which becomes more challenging with the rapid develop- ment of LLMs (Celikyilmaz et al., 2020; Chang et al., 2023). Currently, there are mainly two lines of work on LLM evaluation, including NLU-style and NLG-style evaluations. NLU-style evaluation methods utilize natural language understanding (NLU) tasks such as multi-choice QA to measure the performance of LLMs via simple objective met- rics (such as accuracy and F1 score) (Hendrycks et al., 2021; Zhong et al., 2023; Huang et al., 2023b), which may deviate from the common us- age of LLMs and may not exactly reflect the ability of LLMs in generating responses for user queries.'),
  0.7049670870956175),
 (Document(metadata={'source': '2402.01383v1/2311.00686v1.Little

In [9]:
# Let's call our function we have defined
formatted_response, response_text = query_rag(f'what are recent developments in {topic}?',k=10)
# and finally, inspect our final response!
print(response_text)

 Recent developments in Large Language Model (LLM)-based Natural Language Generation (NLG) evaluation have gained significant attention due to the advancements in LLMs and refined alignment techniques. These models, which include instruction-tuned models, offer great promise as effective and generalized automatic dialogue evaluators. In this response, we will discuss the current status of LLM-based NLG evaluation, its challenges, and recent studies that demonstrate strong correlations between LLMs and human evaluations.

First, let us examine the current state of LLM-based NLG evaluation. The recent advancement in LLMs, such as BERT (Brown et al., 2020), T5 (Raffel et al., 2019), and PaLM (Chowdhery et al., 2022), has led to an extensive array of general-purpose AI assistants capable of tackling a broad spectrum of NLP tasks. Harnessing their strong language understanding capability, LLMs have been used as effective NLG evaluators in various applications, including summarization evalua

In [10]:
print(len(response_text.split(' ')))

864


In [11]:
store_summary(formatted_response,'rag10_large_data') #'rag10'

In [12]:
# Let's call our function we have defined
formatted_response, response_text = query_rag(f'what are recent developments in {topic}?',k=5)
# and finally, inspect our final response!
print(response_text)
store_summary(formatted_response,'rag5_large_data') #rag5

 Recent developments in Large Language Model (LLM) based Natural Language Generation (NLG) evaluation have gained significant attention due to the increasing importance of thoroughly evaluating LLMs given their advanced capabilities. Traditionally, NLP models were assessed using standardized benchmark test suites. However, with the advancement of LLMs, several studies suggest a more comprehensive understanding of model performance by employing a diverse set of NLP benchmarks (Hendrycks et al., 2021; Gao et al., 2021; Srivastava et al., 2022; Liang et al., 2022; Li et al., 2023b).

The gap between users' perception and standardized evaluation suites has been acknowledged in recent studies (Gudibande et al., 2023; Zheng et al., 2023), leading to the incorporation of human evaluations for a more nuanced understanding of model performance. Human evaluations, while valuable, can be costly and time-consuming. To address this challenge, some recent works utilize state-of-the-art LLMs such as 

In [14]:
print(len(response_text.split(' ')))

849


In [33]:
# Let's call our function we have defined
formatted_response, response_text = query_rag(f'what are recent developments in {topic}?',k=8)
# and finally, inspect our final response!
print(response_text)
#store_summary(formatted_response,'rag8')

 Recent developments in Large Language Models (LLMs) have significantly impacted the field of Natural Language Generation (NLG), leading to new research directions and challenges in evaluating NLG systems using LLMs. In this response, we will discuss the current status and challenges of LLM-based NLG evaluation.

Firstly, it is essential to understand that NLG evaluation is a long-standing task in Natural Language Processing (NLP), which has become more challenging with the rapid development of LLMs (Celikyilmaz et al., 2020; Chang et al., 2023). Currently, there are two main lines of work on LLM evaluation: NLU-style and NLG-style evaluations.

NLU-style evaluation methods focus on measuring the ability of an LLM to understand and process natural language instructions accurately (Brown et al., 1995). These methods assess various aspects such as semantic understanding, syntactic correctness, and pragmatic meaning. However, NLU-style evaluations may not fully capture the nuances of open

In [36]:
len(response_text.split(' '))

751

In [38]:
len('Hi what is the concept?'.split(' '))

5

In [39]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:
{context}
 - -
Answer the question based on the above context: {question}
"""

In [40]:
def query_rag(query_text,k=3):
  # Retrieving the context from the DB using similarity search
  results = db.similarity_search_with_relevance_scores(query_text, k=k)

  # Check if there are any matching results or if the relevance score is too low
  if len(results) == 0 or results[0][1] < 0.7:
    print(f"Unable to find matching results.")

  # Combine context from matching documents
  context_text = "\n\n - -\n\n".join([doc.page_content for doc, _score in results])
 
  # Create prompt template using context and query text
  prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
  prompt = prompt_template.format(context=context_text, question=query_text)

  llm = Ollama(model = "mistral",temperature=0)
  # Generate response text based on the prompt
  response_text = llm.invoke(prompt)
 
   # Get sources of the matching documents
  sources = [doc.metadata.get("source", None) for doc, _score in results]
 
  # Format and return response including generated text and sources
  formatted_response = f"Response: {response_text}\nSources: {sources} \nSourceText:{context_text}"
  return formatted_response, response_text

In [41]:
# Let's call our function we have defined
formatted_response, response_text = query_rag(f'what are recent developments in {topic}?',k=8)
# and finally, inspect our final response!
print(response_text)
store_summary(formatted_response,'rag8short')
print(len(response_text.split(' ')))

 Recent developments in LLM-based NLG (Natural Language Generation) evaluation involve the use of Large Language Models (LLMs) as evaluators for NLG systems due to their promising instruction-following and generalization capabilities. Two main lines of research are currently being explored: NLU-style and NLG-style evaluations.

NLU-style evaluation methods focus on assessing the understanding and processing abilities of LLMs, while NLG-style evaluations evaluate the generation quality of LLMs. However, current work often overlooks the rich correlation between various aspects of NLG evaluation, leading to significant differences in evaluation results.

To address this gap, researchers propose an NLG evaluation metric called LLM-Based NLG Evaluation. This approach aims to showcase the versatility and potential of LLMs in improving NLG evaluation methodologies by exploring the synergy between LLMs and humans in establishing evaluation criteria and conducting multi-dimensional evaluations 

In [7]:
s = """superior performance on various NLG evaluation tasks. However, current work often employs the LLM to independently evaluate different aspects, which largely ignores the rich correlation be- tween various aspects. To fill this research gap, in this work, we propose an NLG evaluation metric called"""
k = """LLM-Based NLG Evaluation With the emergence of LLM, recent research works focus on LLM-based evaluators given their promising instruction-following and generalization capability. A first line of work goes through preliminary explorations on LLM-based evaluators, including prompting methods and"""
print(len(s.split(' ')))
print(len(s))
print(len(k))

44
296
294
